# Notebook for "Models in Pyro: From Primitive Distributions to Stochastic Functions"
http://pyro.ai/examples/intro_part_i.html

In this workbook I go through the first example in the Pyro docs, making notes as I go.

In [1]:
import torch
from torch.autograd import Variable
import pyro
import pyro.distributions as dist

Pyro programs are made up of stochastic functions, which are composed to form more complex stochastic functions, aka models. Stochastic functions can be anything that has a __call__() method, so include regular Python functions and methods.

The stochastic functions in pyro.distributions are GPU-accelerated multivariate distributions built on PyTorch, so I should use these wherever possible. Like in PyMC3, custom distributions come from subclassing Distribution.

In [2]:
mu = Variable(torch.zeros(1))
sigma = Variable(torch.ones(1))
x = dist.normal(mu, sigma)
print x

Variable containing:
 1.0331
[torch.FloatTensor of size 1]



In [3]:
log_p_x = dist.normal.log_pdf(x, mu, sigma)
print log_p_x

Variable containing:
-1.4526
[torch.FloatTensor of size 1]



## pyro.sample

This generates a _named_ sample from a stochastic function, and appears to have the syntax:

var = pyro.sample(name, fn, *args)

Apparently this named-ness is somehow useful:

_"Pyro’s backend uses these names to uniquely identify sample statements and change their behavior at runtime depending on how the enclosing stochastic function is being used."_

Currently this is a little mysterious - my best guess is that this is referring to either autograd or to performing different actions based on the values passed to the function (like tt.switch does).

## Simple model

In [17]:
def weather():
    cloudy = pyro.sample('cloudy', 
                         dist.bernoulli, 
                         Variable(torch.Tensor([0.3]))) # still need to wrap consts
    
    cloudy = 'cloudy' if cloudy.data[0] == 1.0 else 'sunny' # torch holds data in Tensor.data
    
    mean_temp = {'cloudy': [55.0], 'sunny': [75.0]}[cloudy]
    sigma_temp = {'cloudy': [10.0], 'sunny': [15.0]}[cloudy]
    
    temp = pyro.sample('temp', dist.normal,
                       Variable(torch.Tensor(mean_temp)),
                       Variable(torch.Tensor(sigma_temp)))
    return cloudy, temp.data[0]

for _ in range(3):
    print weather()

('sunny', 86.05570220947266)
('sunny', 52.917903900146484)
('sunny', 75.6844253540039)


So here the variable name 'cloudy' is not being used, and it's just a coincidence that the pyro.sample variable is called 'cloudy'; it still works fine if you call it something else.

The following line converts integer value from Bernoulli -> string to go into the relevant dicts defined on the following lines.

## Stochastic functions

Pyro can work with conventional Python logic in a really nice way, for instance in the following example:

In [53]:
def geometric(p, t=None):
    if t is None:
        t = 0
    x = pyro.sample("x_{}".format(t), dist.bernoulli, p)
    if torch.equal(x.data, torch.zeros(1)):
        return x
    else:
        return x + geometric(p, t+1)

print(geometric(Variable(torch.Tensor([0.9]))))

Variable containing:
 0
[torch.FloatTensor of size 1]



This looks like it should be computationally expensive, but that depends how much cleverness/optimisation Torch and Pyro are doing out of sight. 

Using conventional Python functions seems more lightweight/easy to work with than needing to define Ops, though I don't know how much extra machinery will be needed when gradients enter the picture, or if heavy numerics are required (e.g. ODE solutions). I will be very impressed if either of these are easy to do.

In [66]:
def normal_product(mu, sigma):
    z1 = pyro.sample("z1", dist.normal, mu, sigma)
    z2 = pyro.sample("z2", dist.normal, mu, sigma)
    y = z1 * z2
    print type(y)
    print mu.data[0]
    return y

def make_normal_normal():
    mu_latent = pyro.sample("mu_latent", dist.normal,
                            Variable(torch.zeros(1)),
                            Variable(torch.ones(1)))
    fn = lambda sigma: normal_product(mu_latent, sigma)
    return fn

make_normal_normal()(Variable(torch.ones(1)))

<class 'torch.autograd.variable.Variable'>
-0.214653119445


Variable containing:
 0.4611
[torch.FloatTensor of size 1]

So so far we've seen that:
- Pyro uses Torch as the computational framework
- We can define normal Python functions to do our sampling (nice!)
- Calling these functions with pyro.sample generates named Torch Variables
- In some currently vague way this will be essential for what we want Pyro to do, and we need to take care to pass pyro.sample unique names whenever it's called (e.g. by using "mystring\_{}".format())
- This has something to do with specifying the joint distribution of the model - is it building a Bayes net?